In [5]:
## Import tkinter 
import tkinter as tk



In [6]:
## Create Admin Window and establish it as part of the main event loop
adminWindow = tk.Tk()
windowMessage = tk.Label(text="Admin Window Introduction Message")
sceneButton = tk.Button(
    frame1,
    text="Scenes",
    width=200,
    height=25,
    bg="gray",
    fg="black",
)
sceneButton.place(x=0, y=0)
elementButton = tk.Button(
    
    text="Elements",
    width=200,
    height=25,
    bg="gray",
    fg="black",
)
elementButton.place(x=0, y=0)
elementButton.pack()

adminWindow.mainloop()

TclError: bad fill style "n": must be none, x, y, or both

: 